In [ ]:
import matplotlib.pyplot as plt
import cv2

from byotrack import Video, VideoTransformConfig

import skimage.io as iio
from byotrack.video.transforms import ChannelSelect, ChannelAvg, ScaleAndNormalize 
from PIL import Image
import numpy as np #can switch this out for pytorch at somepoint - notation is identical


icy_path = "/home/noah/Documents/icy-2.4.2.0-all"
tifpath = '/home/noah/Desktop/cellsegtest/segTestNew/shortStack_adjusted' #path to sequence of tiff files

#for video playback only
fps = 7

In [ ]:
#reader funtion for tif sequences - handles reshaping and normalising (using the stardist recommended normaliser) (doesn't seem to perform that well)
from csbdeep.utils import normalize as csbdeepNormaliser
    
def Read_Data_TIFseq(vid_path):
    # positions = (pd.read_csv(csv_path,usecols=['TrackID','t','x','y'])).values
    vid = iio.ImageCollection(vid_path + '/*.tif').concatenate() #concatonate to numpyarray
    # red_vid = iio.ImageCollection(red_vid_path + '/*.tif')
    vid = vid.reshape(vid.shape[0], vid.shape[1], vid.shape[2], 1)
    normalisedVid = np.asarray([csbdeepNormaliser(frame) for frame in vid])
    return normalisedVid

video = Read_Data_TIFseq(tifpath)

In [ ]:
# Visualisation of the whole video with opencv

for i, frame in enumerate(video):
    try:
        # Display the resulting frame
        cv2.imshow('Frame', frame)
        cv2.setWindowTitle('Frame', f'Frame {i} / {len(video)}')

        # Press Q on keyboard to  exit
        key = cv2.waitKey(1000 // fps) & 0xFF

        if key == ord('q'):
            break

        if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
            break
    except Exception as e:
        print(e)
        cv2.destroyAllWindows()

# Closes all the frames
cv2.destroyAllWindows()
#warning with opencv QOBject::moveToThread can be fixed: https://stackoverflow.com/questions/52337870/python-opencv-error-current-thread-is-not-the-objects-thread

In [ ]:
import numpy as np
import torch

from byotrack.implementation.detector.stardist import StarDistDetector
from byotrack.implementation.detector.wavelet import WaveletDetector

In [ ]:
# Create the detector object with its hyper parameters
model_path = "/home/noah/Desktop/STARDIST_CONFOCAL/NEWEST TdT MODELS/10X_IMAGES_ONLY"
detector = StarDistDetector(model_path, batch_size=5)
#TODO: figure out batch size

In [ ]:
# Test model parameters
detections_sequence_test = detector.run(video[:50])

In [ ]:

#TODO: add roi funtionality to this so plotting shows mask over image (better evaluation) Will need to edit both stardist detector and the detector class .detect() function to include polygon data and add imageJ ROI code
vidCopy = video[0:50] #test batch
scale = 1

global frameID
frameID = 0
frame = video[frameID]
h, w = frame.shape[0:2]
global mask_glob
mask_glob = (detections_sequence_test[frameID].segmentation.numpy() != 0).astype(np.uint8) * 255

# window_name = 'Frame', f'Frame {frameID} / {len(detections_sequence_test)} - Number of detections: {len(detections_sequence_test[i])}'
window_name = 'Paramater Test - Segmentation   (Press Q to Quit)'

try:

    #create and rescale window
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, h*scale, w*scale)

    #Frame Trackbar
    def update_frame(x): #callback function for trackbar - default argument is the position of the track bar
        detections = detector.detect(vidCopy[x][None, ...])
        global mask_glob
        mask_glob = (detections[0].segmentation.numpy() != 0).astype(np.uint8) * 255
    cv2.createTrackbar('Frame',window_name,0,len(vidCopy)-1,update_frame)

    #Probability Trackbar
    def update_probability_threshold(x):
        detector.prob_threshold = x/100
        update_frame(frameID)
    cv2.createTrackbar('Probability Threshold', window_name, 0, 99, update_probability_threshold)

    #Overlap Trackbar
    def update_overlap_threshold(x):
        detector.nms_threshold = x/100
        update_frame(frameID)
    cv2.createTrackbar('Overlap Threshold', window_name, 0, 99, update_overlap_threshold)

except Exception as e:
    print(e)


while True:
    try:

        frameID = cv2.getTrackbarPos('Frame',window_name)
        # cv2.imshow(window_name, video[frameID])
        cv2.imshow(window_name, mask_glob)

        probabilityThreshold = cv2.getTrackbarPos('Probability Threshold', window_name)/100
        nmsThreshold = cv2.getTrackbarPos('Overlap Threshold', window_name)/100

        #exit on q
        if cv2.waitKey(5) == ord('q'):
            # press q to terminate the loop
            cv2.destroyAllWindows()
            break

    except Exception as e:
        print(e)
        cv2.destroyAllWindows()
        break
    
cv2.destroyAllWindows()
print('Prob: ', probabilityThreshold)
print('nms: ', nmsThreshold)

In [ ]:
# Set hyperparameters manually on the video
# Use w/x to move backward/forward in the video
# Use c/v to update k (the main hyperparameter)
# You can restard with another scale/min_area


i = 0
prob_speed = 0.1
nms_speed = 0.2
while True:
    try:
        frame = video[i]
        detections = detector.detect(frame[None, ...])[0][0]
        print(detections)
        mask = (detections.segmentation.numpy() != 0).astype(np.uint8) * 255
        print('a')
        # Display the resulting frame
        cv2.imshow('Frame', mask)
        cv2.setWindowTitle('Frame', f'Frame {i} / {len(video)} - prob={detector.prob_threshold}, nms={detector.nms_threshold} - Num detections: {detections.length}')
        print('b')
        # Press Q on keyboard to  exit
        key = cv2.waitKey() & 0xFF

        if key == ord('q'):
            break

        if cv2.getWindowProperty("Frame", cv2.WND_PROP_VISIBLE) <1:
            break

        if key == ord("a"):
            i = (i - 1) % len(video)

        if key == ord("d"):
            i = (i + 1) % len(video)

        if key == ord("c"):
            detector.prob_threshold = detector.prob_threshold * (1 - prob_speed)

        if key == ord("v"):
            detector.prob_threshold = detector.prob_threshold * (1 + prob_speed)

        if key == ord("b"):
            detector.nms_threshold = detector.nms_threshold * (1 - nms_speed)

        if key == ord("n"):
            detector.nms_threshold = detector.nms_threshold * (1 + nms_speed)

    except Exception as e:
        print(e)
        cv2.destroyAllWindows()
        break

cv2.destroyAllWindows()